Usecase : Read the file movies realted document, decide whether clarification is needed. Summarise the text using the prompt to LLM.

Readme
read_file_tool(state):
Reads a text file from state["filepath"] using UTF‑8.
Returns {"results": file_text, "confidence": 0.95} on success; logs and returns empty results with 0.0 on error.
Needs logger and Path imported.
decide_if_clarify(state):
Routes workflow: returns "clarify" if confidence < CONFIDENCE_THRESHOLD else "summarize".
Needs CONFIDENCE_THRESHOLD and logger defined.
ask_user_tool(state):
Prompts the user for clarification using state["clarify_question"].
Returns {"clarification": "..."}.
Raises if empty input. Uses logger.
llm_summarize_tool(state):
Sends state["results"] to an LLM and returns {"final_summary": summary}.
Runs the pipeline: read -> route -> optional clarify -> summarize -> save -> print final state.
 

In [ ]:
groq_api_key="YOUR_GROQ_API_KEY"


In [55]:
from langchain_groq import ChatGroq

In [63]:
llm = ChatGroq(groq_api_key=groq_api_key,model="llama-3.1-8b-instant")
llm

ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 8192, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x0000024957848D10>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000024957849590>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [64]:
print(llm.invoke("Ping").content)

INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


It seems like you're trying to initiate a conversation or test the connection. 

How can I assist you today?


In [ ]:
from langgraph.graph import StateGraph, END
from typing import Dict

from langchain_groq import ChatGroq
from langchain_core.messages import HumanMessage

from typing import Dict, Optional
import logging
from pathlib import Path
from typing import Dict, Optional
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

CONFIDENCE_THRESHOLD = 0.5          # threshold for “do we need clarification?”
OUTPUT_FILE = "summary_output.txt"


In [71]:
def read_file_tool(state: dict) -> dict:
    """
    Reads the entire contents of a text file whose path is supplied in
    ``state["filepath"]`` and returns it as ``results`` together with a
    high confidence score.

    Expected input state keys:
        - "filepath": str or pathlib.Path – absolute or relative path to the file.

    Returns:
        dict with keys:
            - "results": str – the raw file contents (may be empty if the file is empty)
            - "confidence": float – 0.95 if the file was read successfully,
                                      0.0 otherwise (and an error is logged).
    """
    raw_path = state.get("filepath")
    if not raw_path:
        logger.error("No 'filepath' supplied in state.")
        return {"results": "", "confidence": 0.0}

    try:
        path = Path(raw_path).expanduser().resolve()
        logger.info("Reading source file: %s", path)

        # Read the whole file as UTF‑8 text
        content = path.read_text(encoding="utf-8")
        logger.info("[FILE_READ] %d characters read.", len(content))

        # We give a *high* confidence because the source is deterministic.
        return {"results": content, "confidence": 0.95}
    except Exception as exc:
        logger.exception("Failed to read file %s: %s", raw_path, exc)
        return {"results": "", "confidence": 0.0}

        

In [67]:
def decide_if_clarify(state: dict) -> str:
    """
    Simple router used by the workflow engine.
    Returns the name of the next node:
        - "clarify"   if confidence < CONFIDENCE_THRESHOLD
        - "summarize" otherwise
    """
    confidence = state.get("confidence", 0.0)
    logger.debug("Current confidence: %.2f", confidence)
    return "clarify" if confidence < CONFIDENCE_THRESHOLD else "summarize"


In [73]:
def ask_user_tool(state: dict) -> dict:
    """
    Prompts the user on the console for a clarification string.
    The prompt can be customised via ``state["clarify_question"]``.
    """
    question = state.get("clarify_question", "Could you clarify your request?")
    answer = input(f"\nClarification needed: {question}\nYour answer: ").strip()
    if not answer:
        raise ValueError("Clarification cannot be empty.")
    logger.info("[USER INPUT] %s", answer)
    return {"clarification": answer}

def llm_summarize_tool(state: Dict[str, str]) -> Dict[str, Optional[str]]:
    """Summarise `state['results']` into five sentences."""
    raw_text = state.get("results", "").strip()
    if not raw_text:
        print(f"[SUMMARY] {summary}\n")
        return {"final_summary": ""}

    prompt =("You are a concise policy assistant. Use ONLY the provided text. "
        "Summarize in 5 sentences and cite important terms.\n\n" 
        f"{raw_text}")
    # ← the result is a single AIMessage
    result = llm.invoke([HumanMessage(content=prompt)])
    summary = result.content
    return {"final_summary": summary}

def save_to_file_tool(state: dict) -> dict:
    """
    Writes ``state["final_summary"]`` to ``OUTPUT_FILE`` (or a custom path
    supplied via ``state["output_path"]``) and returns the filename.
    """
    filename = state.get("output_path", OUTPUT_FILE)
    try:
        Path(filename).write_text(state.get("final_summary", ""), encoding="utf-8")
        logger.info("[SAVED] %s", filename)
        return {"file_saved": filename}
    except Exception as exc:
        logger.exception("Failed to write output file %s: %s", filename, exc)
        raise



In [74]:

if __name__ == "__main__":
    # Read the file
    state = {"filepath": "genai_project_assement_input.txt"}          
    state.update(read_file_tool(state))

    #Decide next step
    next_step = decide_if_clarify(state)

    # If we need clarification, ask the user (otherwise skip)
    if next_step == "clarify":
        state.update(ask_user_tool(state))
    # Summarise with the LLM
    state.update(llm_summarize_tool(state))

    # Save the result
    state.update(save_to_file_tool(state))

    # Print final state for debugging (optional)
    print(state)

    


INFO:__main__:Reading source file: C:\Users\rkumbhar\OneDrive - Deloitte (O365D)\genai_project_assement_input.txt
INFO:__main__:[FILE_READ] 774 characters read.
INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:__main__:[SAVED] summary_output.txt


{'filepath': 'genai_project_assement_input.txt', 'results': 'Title: The Shawshank Redemption\nGenre: Drama\nRelease Year: 1994\nRating: 9.3\n\nTitle: The Godfather\nGenre: Crime, Drama\nRelease Year: 1972\nRating: 9.2\n\nTitle: The Dark Knight\nGenre: Action, Crime, Drama\nRelease Year: 2008\nRating: 9.0\n\nTitle: Pulp Fiction\nGenre: Crime, Drama\nRelease Year: 1994\nRating: 8.9\n\nTitle: Inception\nGenre: Action, Sci-Fi, Thriller\nRelease Year: 2010\nRating: 8.8\n\nTitle: Forrest Gump\nGenre: Drama, Romance\nRelease Year: 1994\nRating: 8.8\n\nTitle: Interstellar\nGenre: Adventure, Drama, Sci-Fi\nRelease Year: 2014\nRating: 8.6\n\nTitle: The Matrix\nGenre: Action, Sci-Fi\nRelease Year: 1999\nRating: 8.7\n\nTitle: Parasite\nGenre: Comedy, Drama, Thriller\nRelease Year: 2019\nRating: 8.6\n\nTitle: Gladiator\nGenre: Action, Adventure, Drama\nRelease Year: 2000\nRating: 8.5', 'confidence': 0.95, 'final_summary': "The provided text lists top-rated films across various genres. Here's a conc